# Getting Started

**Vizic** uses MongoDB for data storage, and an added Jupyter server extension to direct catalog request to the database. Therefore the server extension needs to be specified when starting the Jupyter Notebook App from the command line.

To start the Notebook App:

	jupyter notebook --NotebookApp.nbserver_extensions="{'vizic.mongo_ext.extension':True}"

In addition, a running MongoDB instance is requied. To start a new instance on the local machine, enter the following command in a new terminal window:
    
    mongod

On Linux machines, MongoDB instance can be also started with:

    sudo service mongod start

<div class="alert alert-info">

**Note:** 

Before starting a MongoDB instance, the directory of dabase files must exist. The default path is "/data/db/". If such path doesn't exist, errors will be thrown by MongoDB. In some cases, you might experience difficulties starting an instance due to permission issue or file lock, the solution would be specify a new directory for data storage using the command 

    mongod --dbpath /custom/path/

</div>

After the MongoDB database and the Jupyte App are setted up, we can start to work with **Vizic**. Some basic features are deomnstrated below, for more indepth exmaples and a demo dataset, please check out the repository on GitHub.

## Connection

In [ ]:
from vizic import *
import pandas as pd
import numpy as np
from ipywidgets import *

Initiate database connection using the port number (sevrPort, defaults to `8888`) that Jupyter server listens on, here we assume the kernel and the notebook server are running on the same machine, and the hostname and port number for the targe MongoDB instance, where the defaults are `localhost` and `27017` respectively.

In [ ]:
c = Connection(sevrPort=8888)

## AstroMap

We first import the demo catalog from a csv file to a pandas dataframe. Then create a catalog layer using this pandas dataframe. The data used can be obtained from the github repository.

In [ ]:
df_vizic = pd.read_csv('demo.csv')
df_vizic['radius'] = df_vizic.petroR90_r # assign radius column
c.rm_catalog('vizic') # for demo purpose, remove catalog with same given name if existed
g = GridLayer(c, coll_name="vizic", df=df_vizic, scale_r=2) # use scale_r to scale up visualized objects

<div class="alert alert-info">

**Note:** 

To implement the tiled web map design, **vizic** uses the size information to filter out objects that are too small to display at current zoom level. The program uses `B_IMAGE`, which is normally the semi-minor axis of an object, or `RADIUS` as the ruler. If none of the above columns is presented in the dataframe, **vizic** will display all objects regardless of the zoom level with a significant performance cost. 

</div>

<div class="alert alert-info">

**Note:** 
    
The default unit for object sizes is arcsecond.

</div>

After some datasets are successfully imported, we can also retrieve the catalog either by a randomly generated uuid or a customized name provided when the data was initially imported.

In [ ]:
g = GridLayer(c, collection='vizic',scale_r=2)

Now let's create the main map widget (AstroMap) and some control widgets for the catalog layer.

In [5]:
# Map object
m = AstroMap(default_tiles=g, zoom=1) 
# A color picker widget
cp = LayerColorPicker(layer=g, concise=True)
# PopupDis widget to display catalog data individually checked object
p = PopupDis(layer=g)
# A reset button
hb = HomeButton(m)
# A trigger to enter data selection mode
s = SelectionTrig(m); s.link()
# A button to initiate query for selected from the database
get = GetDataButton(g)

In [6]:
# Create a dropdown widget to select the property used for color-mapping
cdrop = CFDropdown(g)
g.custom_c = True
# Create a widget to filter displayed objects with given value ranges
f = FilterWidget(g)
g.filter_obj = True
f.link()

Use an accordion widget to put control widgets together.

In [7]:
accordion = Accordion(children=[cdrop, f, p])
accordion.set_title(2, 'Object Catalog')
accordion.set_title(0, 'Color by Property')
accordion.set_title(1, 'Filter Objects')
accordion.layout.width = '50%'
accordion.layout.overflow_x = 'scroll'

Put the accordion side by side with the map widget to make an integrated interface.

In [8]:
app = HBox([m, VBox([hb, s, get, cp]), accordion]); app

## Interface and Interaction

### Basic map browsing

![](./gifs/browse.gif)

### Color mapping & objects filtering

![](./gifs/interaction.gif)

### Clickable objects

![](./gifs/popup.gif)

### Selection tool

![](./gifs/selection.gif)